## Larger C or C++ modules ... different method from ctypes
Documentation https://docs.python.org/3/extending/extending.html

  sudo apt-get install python3-dev  # for python3.x installs

* Begin by creating a file mymodule.c. 
* The first two lines of the file are
* get the Python API 
https://github.com/python/cpython/blob/main/Include/Python.h
(https://docs.python.org/3/c-api/arg.html#strings-and-buffers)

https://svn.python.org/projects/python/trunk/Modules/xxmodule.c (example)

    #define PY_SSIZE_T_CLEAN 
    #include <Python.h>

    static PyObject *
    fn_system(PyObject *self, PyObject *args)
    {
        const char *command;
        int sts;

        if (!PyArg_ParseTuple(args, "s", &command))
            return NULL;
        sts = system(command);
        return PyLong_FromLong(sts);
    }
    
The C function always has two arguments, conventionally called self (module or object) and args (pointer to a Python tuple object containing the arguments).

The function PyArg_ParseTuple() in the Python API checks the argument types and converts them to C values. 
PyArg_ParseTuple() returns true (nonzero) if all arguments have the right type and its components have been stored in the variables whose addresses are passed. It returns false (zero) if an invalid argument list was passed, and raises an exception so the calling function can return NULL immediately .



To call the function from Python we need to list its name and address in a method table:

    static PyMethodDef MyMethods[] = {
        
        {"system",  fn_system, METH_VARARGS,
         "Execute a shell command."},
        
        {NULL, NULL, 0, NULL}        /* Sentinel */
    };

The module definition structure (referring to the method table)

    static struct PyModuleDef mymodule = {
        PyModuleDef_HEAD_INIT,
        "mymodule",   /* name of module */
         my_doc, /* module documentation, may be NULL */
        -1,       /* size of per-interpreter state of the module,
                     or -1 if the module keeps state in global variables. */
        MyMethods
    };

Module initialisation

        PyMODINIT_FUNC
        PyInit_mymodule(void)
        {
            return PyModule_Create(&mymodule);
        }

    int main(int argc, char *argv[])
    {
        wchar_t *program = Py_DecodeLocale(argv[0], NULL);
        if (program == NULL) {
            fprintf(stderr, "Fatal error: cannot decode argv[0]\n");
            exit(1);
        }

        /* Add a built-in module, before Py_Initialize */
        if (PyImport_AppendInittab("spam", PyInit_mymodule) == -1) {
            fprintf(stderr, "Error: could not extend in-built modules table\n");
            exit(1);
        }

        /* Pass argv[0] to the Python interpreter */
        Py_SetProgramName(program);

        /* Initialize the Python interpreter.  Required.
           If this step fails, it will be a fatal error. */
        Py_Initialize();

        /* Optionally import the module; alternatively,
           import can be deferred until the embedded script
           imports it. */
        PyObject *pmodule = PyImport_ImportModule("mymodule");
        if (!pmodule) {
            PyErr_Print();
            fprintf(stderr, "Error: could not import module 'mymodule'\n");
        }

        PyMem_RawFree(program);
        return 0;
    }

then sudo python3 setup.py install https://docs.python.org/3/extending/building.html#building


To make the new module a permanent part of the Python interpreter, you change the configuration setup and rebuild the interpreter.On linux: just place your file (mymodule.c for example) in the Modules/ directory of an unpacked source distribution, add a line to the file Modules/Setup.local describing your file:

    mymodule mymodule.o

rebuild the interpreter by running make in the toplevel directory. You can also run make in the Modules/ subdirectory, but then you must first rebuild Makefile there by running ‘make Makefile’. (This is necessary each time you change the Setup file.)